## Scrape Zillow Housing Data
- Get all the housing information for renting location around UCSD, i.e. in La Jolla 

Part 1: From General Overall Page
- bs4 + request; maybe a little bit of automation to interact with search bar and bottoms
- Data Features: 
    1. Price
    2. Address
    3. beds, baths, sqft 

Part 2: From Individual Listing Page  

In [1]:
import requests 
from bs4 import BeautifulSoup as soup

In [2]:
# sample url for La Jolla renting list
url = "https://www.zillow.com/homes/La-Jolla-_rb/"

In [3]:
# taken header from browser http request
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.44',
    'refer': 'https://www.zillow.com/homes/San-Diego-_rb/'}

In [10]:
# get the website 
html = requests.get(url, headers=header) # don't to run it too much!!!
print(html.status_code)

200


In [12]:
# save it for later experiment
import os
if os.path.exists('zillow_la_jolla.txt'): os.remove('zillow_la_jolla.txt')
with open('zillow_la_jolla.txt', 'w') as file:
    file.write(html.text)

In [20]:
# function to read HTML file back to string and parse it using bs4
def parse_file(fp):
    with open(fp, mode='r') as file:
        html_txt = file.readlines()
    return soup(''.join(html_txt), 'html.parser')
html_txt = parse_file('zillow_la_jolla.txt')

In [69]:
# extract price, and detail info about the house
import re 
pattern=r"([1-9] bd)[s]*([1-9] ba)([0-9,\-]+ sqft)-(.*)"

info=[]
for house in html_txt.findAll('div', {'class':'list-card-heading'}):
    if len(house.text)==0: # if there is nothing (usually happens at the end)
        print('This is the end of the page')
        continue 
    ins={}
    price=house.find('div', {'class': 'list-card-price'}).text
    detail=house.find('ul', {'class': 'list-card-details'}).text
    match=re.match(pattern, detail)
    for index, cat in enumerate(['bds','ba','sqft','house-type']): 
        ins[cat]=match.group(index+1)
    info.append(info)

This is the end of the page


In [ ]:
# extract the address of the house 
